In [ ]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import math
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters
from adam.geometry import linkParametric
from urdfpy import xyz_rpy_to_matrix, matrix_to_xyz_rpy

In [ ]:
def SX2DM(sx):
    return cs.DM(sx)

In [ ]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['r_hip_3']
r_hip_r_char = linkParametric.LinkCharacteristics(0.0)
r_hip_yaw_char = linkParametric.JointCharacteristics(0.1451)

link_characteristics = {'r_hip_3': r_hip_r_char}
joint_characteristics = {'r_hip_yaw': r_hip_yaw_char}
print(joint_characteristics.items())
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link, link_characteristics)

In [ ]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =np.random.rand(len(link_name_list))  
lenght_multiplier = np.random.rand(len(link_name_list))
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

In [ ]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()
print("M", M(H_b, s, density, lenght_multiplier))
print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

In [ ]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

In [ ]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

In [ ]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

In [ ]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

In [ ]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))

In [ ]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

In [ ]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

In [ ]:
link_original = kinDyn.get_element_by_name('r_hip_3', kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(0.0)
print(link_original.visuals[0].geometry.cylinder)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume
print('radius\n', radius)
print('lenggth\n', length)
print('origin\n',matrix_to_xyz_rpy(link_original.inertial.origin))
# print('volum\n', volume)
print('mass\n', mass)
# print('density\n', density_num)
print('inertia\n', link_original.inertial.inertia)
link_parametric =linkParametric.linkParametric('r_hip_3', 1.0,density_num, kinDyn.robot, link_original, r_hip_r_char)
print('Link Parametric')
print('radius \n', link_parametric.visual_data_new[1])
print('length\n', link_parametric.visual_data_new[0])
# print('origin\n', link_parametric.origin)
# print('volume\n', link_parametric.volume)
print('mass\n', link_parametric.mass)
print('density\n', link_parametric.density)
print('inertia\n', link_parametric.I.ixx)
print('inertia\n', link_parametric.I.iyy)
print('inertia\n', link_parametric.I.izz)